In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *

from fastai.text import *

### Hindi Sentiment Analysis

In [ ]:
lang = 'hi'
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
data_path = Config.data_path(); data_path
name = f'{lang}wiki'
path = data_path/name

## BBC Articles(Hindi)

#### Language Model

We are gonna perform sentiment analysis on 2 datasets: BBC news articles,IIT-P movie reviews and IIT-P product reviews. The datasets can be found [here](https://github.com/AI4Bharat/indicnlp_corpus#publicly-available-classification-datasets).

In [27]:
train_df = pd.read_csv('data/bbc-articles-hi/hi-train.csv', header=None)
train_df.head()

,0,1
0,india,मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से...
1,pakistan,नेटिजन यानि इंटरनेट पर सक्रिय नागरिक अब ट्विटर...
2,news,इसमें एक फ़्लाइट एटेनडेंट की मदद की गुहार है औ...
3,india,"प्रतीक खुलेपन का, आज़ाद ख्याली का और भीड़ से अ..."
4,india,ख़ासकर पिछले 10 साल तक प्रधानमंत्री रहे मनमोहन...


In [28]:
test_df = pd.read_csv('data/bbc-articles-hi/hi-test.csv', header=None)
test_df.head()

,0,1
0,india,बुधवार को राज्य सभा में विपक्ष के सवालों के जव...
1,india,लखनऊ स्थित पत्रकार समीरात्मज मिश्र को बुलंदशहर...
2,india,लगभग 1300 हेक्टेयर ज़मीन का अधिग्रहण किया जा च...
3,international,हालांकि उनके अंगरक्षकों को बमों को जाम करने वा...
4,india,आयोग का कहना है कि इस तरह के परीक्षण से महिलाओ...


In [29]:
df = pd.concat([train_df,test_df], sort=False)

In [58]:
data_lm = (TextList.from_df(df, path, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [37]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [38]:
lr = 1e-3
lr *= bs/48

In [39]:
# finetuning the language model
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.429507,4.066370,0.291303,01:06
1,4.114826,3.962841,0.301713,01:06


In [40]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.925475,3.930910,0.305314,01:07
1,3.882566,3.891310,0.309718,01:06
2,3.819858,3.862531,0.313200,01:06
3,3.753082,3.844485,0.315466,01:06
4,3.677970,3.832285,0.317484,01:06
5,3.621572,3.827234,0.318892,01:06
6,3.591659,3.825799,0.319171,01:06
7,3.590563,3.827231,0.319184,01:06


In [41]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

In [65]:
learn_lm = None
learn = None
gc.collect()

42

### Classifier

In [42]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols=1)
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [13]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [67]:
data_clas.classes, data_clas.c

(['business',
  'china',
  'entertainment',
  'india',
  'institutional',
  'international',
  'learningenglish',
  'multimedia',
  'news',
  'pakistan',
  'science',
  'social',
  'southasia',
  'sport'],
 14)

In [14]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1), average='micro')

mcc = MatthewsCorreff()

In [15]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1, mcc]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [16]:
lr=2e-2
lr *= bs/48

In [17]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,1.198869,1.184566,0.552023,0.552023,0.381528,00:23
1,0.869197,0.722998,0.748555,0.748555,0.653513,00:21


In [18]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.704260,0.755434,0.763006,0.763006,0.679601,00:22
1,0.659796,0.586382,0.820809,0.820809,0.759722,00:23


In [19]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.665057,0.687979,0.791907,0.791907,0.723031,00:26
1,0.567432,0.567387,0.823699,0.823699,0.768312,00:26


In [20]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.416406,0.601812,0.809249,0.809249,0.746467,00:35
1,0.328328,0.576490,0.820809,0.820809,0.759892,00:39


In [21]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,matthews_correff,time
0,0.186924,0.615390,0.832370,0.832370,0.776217,00:46
1,0.160297,0.629750,0.838150,0.838150,0.784047,00:44


In [22]:
learn_c.save(f'{lang}clas')

In [25]:
learn_c.load(f'{lang}clas');

RNNLearner(data=TextClasDataBunch;

Train: LabelList (3121 items)
x: TextList
xxbos मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से नोएडा जाने का समय काफी कम हो जाएगा और यात्रियों को राजीव चौक या मंडी हाउस से होकर नहीं जाना पड़ेगा . लेकिन , यह xxunk लाइन इसलिए भी महत्वपूर्ण है क्योंकि इस पर xxunk यानी बिना ड्राइवर वाली मेट्रो चलाने की योजना है . ऐसा भारत में पहली बार होगा जब कोई मेट्रो बिना ड्राइवर के चलाई जाएगी . मेट्रो के तीसरे xxunk में भारत में पहली बार ड्राइवरलेस तकनीक आएगी लेकिन दुनिया भर में कई देशों में ड्राइवरलेस मेट्रो पहले से ही चलती हैं . इन देशों में ड्राइवरलेस मेट्रो सफल भी रही हैं . दक्षिण कोरिया की राजधानी सोल में xxunk ट्रेन कामयाबी से चल रही है . ये मेट्रो ज़मीन के नीचे चलती है इसमें ड्राइवर का केबिन भी नहीं होता है . यूरोप में डेनमार्क , स्पेन , इटली , फ्रांस , जर्मनी , हंगरी , स्विट्जरलैंड और ब्रिटेन में भी xxunk मेट्रो चलती है . इन देशों में एक से ज़्यादा शहरों में भी ऐसी मेट्रो चलाई जाती है . इनके अलावा अमेरिका और कनाडा में भी ड्राइवरलेस ट्रेन चलती है . वहीं , ब्रा

In [59]:
data_valid = (TextList.from_df(test_df, path, vocab=data_lm.vocab, cols=1)
    .no_split()
    .label_from_df(cols=0)
    .databunch(bs=bs, num_workers=1))

/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py:183: UserWarning: `no_split` is deprecated, please use `split_none`.
  warn("`no_split` is deprecated, please use `split_none`.")


In [60]:
data_valid

TextClasDataBunch;

Train: LabelList (866 items)
x: TextList
xxbos बुधवार को राज्य सभा में विपक्ष के सवालों के जवाब में मनमोहन सिंह ने कहा , " अगर किसी तरह की गड़बड़ी पाई जाती है , तो हम कड़ी से कड़ी कार्रवाई करेंगे . "मंगलवार को संसद में भारत के नियंत्रक और महालेखा परीक्षक की रिपोर्ट पेश की गई है . किसानों के क़र्ज़ की माफ़ी स्कीम पर तैयार इस रिपोर्ट में कहा गया है कि पिछले वित्तीय वर्ष में जितने भी लोगों के क़र्ज़ बैंकों ने माफ़ किए थे , उनमें से 22 फ़ीसद मामले फर्ज़ी थे . रिपोर्ट में कहा गया है कि बैंक के कर्मचारियों ने फर्ज़ी खातों में कर्ज़ माफ़ किए हैं और वो सारी राशि ख़ुद xxunk ली है . विपक्ष ने मामले पर सरकार को घेरना शुरू कर दिया है . मुख्य विपक्षी दल भारतीय जनता पार्टी के प्रवक्ता प्रकाश जावडेकर ने कहा है कि इस तरह की गड़बड़ी इतने बड़े पैमाने पर सिर्फ़ तकनीकी कमियों की वजह से नहीं हो सकती है . विपक्ष और सरकार को समर्थन दे रहे xxunk जैसे समाजवादी पार्टी मामले को संसद में भी उठा रही है . लोकसभा में विपक्ष की नेता सुषमा स्वराज ने अध्यक्ष से मामले पर बहस करवाने की मांग की है . मह

In [61]:
%%time
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(test_df[1]), 'actual_label': list(test_df[0]), 'predicted_label': ['']*test_df.shape[0]}
all_nodes = list(set(train_df[0]))
for node in all_nodes:
    df_dict[node] = ['']*test_df.shape[0]
    
i2c = {}
for key, value in learn_c.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
for index, row in df_result.iterrows():
    pred = learn_c.predict(data_valid.train_ds[index])
    for node in all_nodes:
        row[node] = pred[2][learn_c.data.c2i[node]].item()
    row['predicted_label'] = i2c[pred[0].data.item()]
df_result.head()

,query,actual_label,predicted_label,india,international,business,entertainment,news,pakistan,sport,china,learningenglish,southasia,social,multimedia,institutional,science
0,बुधवार को राज्य सभा में विपक्ष के सवालों के जव...,india,india,0.99837,0.000105902,0.000898691,1.32681e-05,0.000352362,1.24156e-05,0.000142646,1.69676e-06,2.2648e-08,8.80532e-05,5.61723e-07,1.16179e-05,1.0053e-06,1.78515e-06
1,लखनऊ स्थित पत्रकार समीरात्मज मिश्र को बुलंदशहर...,india,india,0.999609,9.95603e-05,1.72562e-06,1.24778e-06,1.65007e-05,1.79436e-06,0.000261029,3.25504e-07,1.8079e-08,2.4526e-06,2.64633e-07,5.35698e-06,2.97535e-07,9.23659e-07
2,लगभग 1300 हेक्टेयर ज़मीन का अधिग्रहण किया जा च...,india,india,0.998489,0.000125899,5.74076e-06,0.000591447,0.00049142,2.80877e-06,5.12592e-06,1.60039e-06,1.72318e-08,1.93073e-05,5.42383e-07,0.000259593,1.71028e-06,5.67388e-06
3,हालांकि उनके अंगरक्षकों को बमों को जाम करने वा...,international,international,0.000194536,0.999052,1.77967e-05,9.84898e-05,0.000104535,7.7294e-06,5.4925e-05,1.87943e-06,1.31205e-06,3.55316e-05,3.26007e-06,0.000300275,4.02626e-05,8.70869e-05
4,आयोग का कहना है कि इस तरह के परीक्षण से महिलाओ...,india,india,0.782914,0.148549,0.00140876,0.0106771,0.0107084,0.00477509,0.0064068,0.00356244,8.02568e-05,0.0213587,0.0012408,0.00648865,0.00087472,0.000955079


In [64]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 17 columns):
query              866 non-null object
actual_label       866 non-null object
predicted_label    866 non-null object
india              866 non-null object
international      866 non-null object
business           866 non-null object
entertainment      866 non-null object
news               866 non-null object
pakistan           866 non-null object
sport              866 non-null object
china              866 non-null object
learningenglish    866 non-null object
southasia          866 non-null object
social             866 non-null object
multimedia         866 non-null object
institutional      866 non-null object
science            866 non-null object
dtypes: object(17)
memory usage: 115.1+ KB


In [62]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7979214780600462

In [63]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.7258121928549706

In [69]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average='micro')

0.7979214780600463